# Context-Aware Content Recommendation System

## Selected Project Track
Artificial Intelligence & Machine Learning – Recommendation Systems

markdown
Copy code
## Problem Definition & Objective

Digital platforms provide users with a massive amount of content such as movies, videos, and articles. This leads to information overload, making it difficult for users to discover content that is both relevant and suitable for their current context.

Traditional recommendation systems often focus only on similarity or historical behavior, ignoring contextual factors such as user mood or time availability.

### Objective
The objective of this project is to build a Context-Aware Content Recommendation System that:
- Recommends relevant content using semantic similarity
- Incorporates user context such as mood and available time
- Improves personalization and user experience

## Problem Statement

To design and implement a content-based recommendation system that analyzes movie metadata and recommends semantically similar movies while adapting recommendations based on contextual constraints such as user mood and time availability.

## Real-World Relevance & Motivation

Content recommendation systems are widely used in real-world applications such as:
- Netflix and Amazon Prime for movie recommendations
- YouTube for video suggestions
- Spotify for music recommendations

Context-aware recommendation systems enhance user engagement by providing recommendations that align with a user’s current situation, leading to improved satisfaction and retention.

## Data Understanding & Preparation

### Dataset Used
The MovieLens dataset is a publicly available benchmark dataset widely used in recommendation system research.

### Files Used
- movies.csv: Contains movie titles and genres
- tags.csv: Contains user-generated tags describing movies

### Preprocessing Steps
- Removal of year information from movie titles
- Cleaning and formatting genre data
- Handling missing tag values
- Merging movie metadata and tags
- Creating a unified textual content feature

In [14]:
!pip install -q sentence-transformers


In [15]:
import pandas as pd
import numpy as np
import random

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

movies = pd.read_csv("/content/movies.csv")
tags = pd.read_csv("/content/tags.csv")

movies.head()



,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [16]:
movies['title'] = movies['title'].str.replace(r'\(\d{4}\)', '', regex=True)
movies['genres'] = movies['genres'].str.replace('|', ' ', regex=False)

tags['tag'] = tags['tag'].astype(str)
tags_grouped = tags.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)).reset_index()

movies = movies.merge(tags_grouped, on='movieId', how='left')
movies['tag'] = movies['tag'].fillna('')

movies['content'] = movies['title'] + ' ' + movies['genres'] + ' ' + movies['tag']

movies.head()


,movieId,title,genres,tag,content
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,children Disney animation children Disney Disn...,Toy Story Adventure Animation Children Comedy...
1,2,Jumanji,Adventure Children Fantasy,Robin Williams fantasy Robin Williams time tra...,Jumanji Adventure Children Fantasy Robin Will...
2,3,Grumpier Old Men,Comedy Romance,comedinha de velhinhos engraÃƒÂ§ada comedinha ...,Grumpier Old Men Comedy Romance comedinha de ...
3,4,Waiting to Exhale,Comedy Drama Romance,characters slurs based on novel or book chick ...,Waiting to Exhale Comedy Drama Romance charac...
4,5,Father of the Bride Part II,Comedy,Fantasy pregnancy remake family Steve Martin s...,Father of the Bride Part II Comedy Fantasy pr...


## Model / System Design

### Approach Used
This project uses a content-based recommendation approach enhanced with context awareness.

### Techniques Applied
- Sentence Embeddings using a pretrained transformer model
- Cosine similarity for semantic similarity measurement
- Context-based filtering for personalization

### System Workflow
1. Preprocess movie metadata
2. Generate sentence embeddings for movie content
3. Compute semantic similarity between movies
4. Apply contextual filters
5. Generate top-N recommendations

In [28]:
movies = movies.sample(100, random_state=42).reset_index(drop=True)

movies['estimated_time'] = random.choices([90, 120, 150], k=len(movies))
movies['mood'] = random.choices(['Light', 'Serious'], k=len(movies))

movies['content'] = movies['content'].str.slice(0, 300)

model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')

embeddings = model.encode(
    movies['content'].tolist(),
    batch_size=64,
    show_progress_bar=False
)


## Core Implementation

Sentence embeddings are used to capture semantic meaning of movie content. Cosine similarity is computed on-demand to generate recommendations efficiently without memory overhead.


In [35]:
def recommend_movies(movie_title, mood=None, max_time=None, top_n=5):
    matched_movies = movies[movies['title'].str.contains(movie_title, case=False, na=False)]

    if matched_movies.empty:
        return "Movie not found in dataset"

    idx = matched_movies.index[0]
    similarity_scores = cosine_similarity([embeddings[idx]], embeddings).flatten()
    similar_indices = similarity_scores.argsort()[::-1][1:]

    recommendations = movies.iloc[similar_indices]

    if mood:
        recommendations = recommendations[recommendations['mood'] == mood]
    if max_time:
        recommendations = recommendations[recommendations['estimated_time'] <= max_time]

    return recommendations[['title', 'genres', 'estimated_time', 'mood']].head(top_n)



In [36]:
recommend_movies("Wolf Creek", mood="Serious", max_time=120)





,title,genres,estimated_time,mood
47,Murderer,Crime Drama Mystery Thriller,90,Serious
82,A Deadly Encounter,Thriller,90,Serious
16,Blood Money,Drama Thriller,90,Serious
1,Face of Evil,Drama Thriller,120,Serious
46,Payday,Action Drama Mystery Thriller,90,Serious


## Evaluation & Analysis

### Observations
- Semantically similar movies are successfully recommended
- Context-aware filtering improves relevance
- Sentence embeddings capture deeper meaning than traditional TF-IDF

### Limitations
- No collaborative filtering
- Context attributes are simulated


## Ethical Considerations & Responsible AI

- No personal or sensitive user data was used
- The system is transparent and explainable
- Bias may exist due to dataset limitations
- Recommendations are generated responsibly


## Conclusion & Future Scope

### Conclusion
A context-aware content recommendation system was successfully implemented using sentence embeddings and cosine similarity, providing semantically meaningful and personalized recommendations.

### Future Scope
- Integrate collaborative filtering
- Use real user interaction data
- Deploy as a web application
- Enhance context modeling
